In [11]:
import pandas as pd
pd.set_option('display.max_columns',None)

In [12]:
import sys
sys.path.append("..")  # 상위 디렉토리를 sys.path에 추가

import requests
import pandas as pd
from mySetting import Client_ID, Client_Secret

# 네이버 API 키
client_id = Client_ID  # 클라이언트 ID
client_secret = Client_Secret  # 클라이언트 Secret

def get_coordinates(address):
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'query': address,
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data['addresses']:
        x = data['addresses'][0]['x']
        y = data['addresses'][0]['y']
        return (x, y)
    else:
        return (None, None)

# 병원

In [13]:
all_hospital = pd.read_excel('../new_data/의료시설/1.병원정보서비스 2024.6.xlsx')

In [16]:
tmp = all_hospital.loc[all_hospital['주소'].str.contains('평택시')][['요양기관명','주소','종별코드명']].reset_index().drop(columns=['index']).rename(columns={'요양기관명':'이름'})
tmp['종별코드명'].value_counts()

종별코드명
의원       308
치과의원     187
한의원      114
병원        12
보건진료소     11
요양병원      10
보건지소       9
한방병원       5
종합병원       4
보건소        2
조산원        1
Name: count, dtype: int64

In [17]:
tmp = all_hospital.loc[all_hospital['주소'].str.contains('평택시')][['요양기관명','주소','종별코드명']].reset_index().drop(columns=['index']).rename(columns={'요양기관명':'이름'})
tmp['위도'] = pd.NA
tmp['경도'] = pd.NA

for i, address in enumerate(tmp['주소']):
    lng, lat = get_coordinates(address)
    tmp.at[i, '위도'] = lat
    tmp.at[i, '경도'] = lng

In [18]:
clinic = tmp.loc[tmp['종별코드명'] == '의원'].drop(columns=['종별코드명'])
oriental = tmp.loc[(tmp['종별코드명'] == '한의원') | (tmp['종별코드명'] == '한방병원')].drop(columns=['종별코드명'])
dentistry = tmp.loc[tmp['종별코드명'] == '치과의원'].drop(columns=['종별코드명'])
hospital = tmp.loc[tmp['종별코드명'] == '병원'].drop(columns=['종별코드명'])
nursing = tmp.loc[tmp['종별코드명'] == '요양병원'].drop(columns=['종별코드명'])
general = tmp.loc[tmp['종별코드명'] == '종합병원'].drop(columns=['종별코드명'])
public = tmp.loc[tmp['종별코드명'].str.contains('보건')].drop(columns=['종별코드명'])

In [19]:
clinic.to_csv('../preprocessed/의료시설/의원.csv', index=False)
oriental.to_csv('../preprocessed/의료시설/한의원.csv', index=False)
dentistry.to_csv('../preprocessed/의료시설/치과.csv', index=False)
hospital.to_csv('../preprocessed/의료시설/일반병원.csv', index=False)
nursing.to_csv('../preprocessed/의료시설/요양병원.csv', index=False)
general.to_csv('../preprocessed/의료시설/종합병원.csv', index=False)
public.to_csv('../preprocessed/의료시설/보건병원.csv', index=False)

# 약국

In [8]:
all_pharmacy = pd.read_excel('../new_data/의료시설/2.약국정보서비스 2024.6.xlsx')
all_pharmacy.head(1)

,암호화요양기호,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,읍면동,우편번호,주소,전화번호,개설일자,좌표(X),좌표(Y)
0,JDQ4MTg4MSM1MSMkMSMkMCMkMDMkNTgxOTYxIzQxIyQxIy...,이층약국,81,약국,110000,서울,110015,은평구,역촌동,3421.0,"서울특별시 은평구 연서로 9, 2층 205(상가)호 (역촌동, 센타폴리스)",02-6085-0466,20180207,126.915668,37.600118


In [9]:
pharmacy = all_pharmacy.loc[all_pharmacy['주소'].str.contains('평택시')][['요양기관명','주소']].reset_index().drop(columns=['index']).rename(columns={'요양기관명':'이름'})
pharmacy['위도'] = pd.NA
pharmacy['경도'] = pd.NA

for i, address in enumerate(pharmacy['주소']):
    lng, lat = get_coordinates(address)
    pharmacy.at[i, '위도'] = lat
    pharmacy.at[i, '경도'] = lng

In [10]:
pharmacy.to_csv('../preprocessed/의료시설/약국.csv')